In [11]:
"""Code assumes the ability to train using a GPU with CUDA.
"""
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from advertorch.attacks import GradientSignAttack, CarliniWagnerL2Attack, PGDAttack
import matplotlib.pyplot as plt

# makes default tensor a CUDA tensor so GPU can be used
torch.set_default_tensor_type('torch.cuda.FloatTensor')

TypeError: type torch.cuda.FloatTensor not available. Torch not compiled with CUDA enabled.

### Define data loaders and data preprocessing steps

In [8]:
data_preprocess = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))])
# the mean of mnist pixel data is .1307 and the stddev is .3081

train_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=True, download=True,
                         transform=data_preprocess), 
                    batch_size=64, 
                    shuffle=True)

test_loader = torch.utils.data.DataLoader(
                    torchvision.datasets.MNIST('./data', train=False, download=True,
                         transform=data_preprocess), 
                    batch_size=1000, 
                    shuffle=True)

torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64

KeyboardInterrupt: 

### Define the model

In [53]:
class LeNet(nn.Module):
    """MNIST-modified LeNet-5 model.
    """
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc1_drop = nn.Dropout(p=.50)
        self.fc2 = nn.Linear(120, 84)
        self.fc2_drop = nn.Dropout(p=.50)
        self.fc3 = nn.Linear(84,10)
        
    def weights_init(m):
        if isinstance(m, nn.Conv2d):
            xavier(m.weight.data)
            xavier(m.bias.data)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)
        x = self.fc1_drop(F.relu(self.fc1(x)))
        x = self.fc2_drop(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

### Define adversarial example generation function

In [54]:
def generate_adversarial_samples(og_samples, true_labels, adversary, num_per_samp=1):
    """Create num_per_samp adversarial examples for each sample in
    og_samples. Return the generated samples along with corresponding 
    adv_labels, a tensor containing the adversarial examples' labels.
    """
    adv_samples = []
    for i in range(num_per_samp):
        adv_samples.append(adversary.perturb(og_samples, true_labels))
    adv_samples = torch.cat(adv_samples, 0)
    adv_labels = torch.cat([true_labels]*num_per_samp, 0)
    return adv_samples, adv_labels

### Define my loss function


In [9]:
def my_loss(output, labels, alpha_wd=0, alpha_jr=0, x=None):
    """Adds terms for L2-regularization and the norm of the input-output 
    Jacobian to the standard cross-entropy loss function. Check https://arxiv.org/abs/1908.02729
    for alpha_wd, alpha_jr suggestions.
    """
    # standard cross-entropy loss base
    loss = F.cross_entropy(output, labels)
    
    # add l2 regularization to loss 
    if alpha_wd != 0:
        l2 = 0
        for p in lenet.parameters():
            l2 += p.pow(2).sum()
        loss = loss + alpha_wd*l2
    
    # new input-output jacobian regularization formulation
    print(output.shape)
    if alpha_jr != 0:
        
        x = x.repeat(1, output.shape[1]).reshape(x_.shape[0]*n_out, x_.shape[1])
    
    # add input-output jacobian regularization to loss 
    if alpha_jr != 0:
        n_classes = output.shape[1]
        j = torch.zeros(output.shape[0], n_classes, x.shape[1], x.shape[2], x.shape[3])
        for i in range(n_classes):
            # for each training example we have the gradient of each output class w.r.t each 
            # input pixel, so shape is (1,10,1,28,28). For each batch, it's (64,10,1,28,28)
            j[:, i] = torch.autograd.grad(output[:, i].sum(), x, retain_graph=True)[0].data
        j = j.view(output.shape[0], -1)
        j = torch.mean(j, dim=0, keepdim=True)
        j = torch.norm(j, dim=1)
        loss = loss + alpha_jr*j

    return loss

### Define train and test functions 

In [56]:
def train(epoch, alpha_wd, alpha_jr, adversary=None):
    lenet.train()
    
    for batch_idx, (samples, labels) in enumerate(train_loader):
        # send inputs and labels to GPU
        samples, labels = samples.to(device), labels.to(device)
        
        # expand dataset with adversarial examples if adversary specified
        if adversary != None:
            adv_samples, adv_labels = generate_adversarial_samples(samples, labels, adversary)  
            samples, labels = torch.cat([samples, adv_samples], 0), torch.cat([labels, adv_labels], 0)
            samples, labels = samples.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # needed so that we can get gradient of output w.r.t input
        samples = Variable(samples, requires_grad=True)
        
        output = lenet(samples)
        loss = my_loss(output, labels, alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} \tLoss: {loss.item():.6f}')
            train_losses.append(loss.item())

In [57]:
def test(alpha_wd, alpha_jr):
    lenet.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for samples, labels in test_loader:
            samples, labels = samples.to(device), labels.to(device)
            output = lenet(samples)
            # negative log-likelihood loss
            test_loss += my_loss(output, labels, alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples).item()
            # output is a tensor, .data retrieves its data, max returns the index of the highest valued element
            preds = output.data.max(1, keepdim=True)[1]
            correct += preds.eq(labels.data.view_as(preds)).sum().item()
                
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * float(correct / len(test_loader.dataset))
    
    print(f'Test set: \n\tAvg. loss: {test_loss:.6f}\n\tAccuracy: ({test_accuracy:.2f}%)')
    
    test_accuracies.append(test_accuracy)
    test_losses.append(test_loss)

### Training

In [10]:
# training details
torch.manual_seed(1)
n_epochs = 30
log_interval = 200
training_run = 1

# varying values for certain hyperparameters to produce models with varying degrees of robustness
epsilons = [0, .1, .2, .3, .4, .5, .6, .7]
alpha_wds = [.000001, .000005, .00001, .00005, .0001, .0005, .001]
alpha_jrs = [.0000001, .000001, .00001, .0001, .001, .01, .1]

performance = {}

for hyp_param_to_vary in [alpha_jrs]:
    epsilon = 0
    alpha_wd = 0
    alpha_jr = 0
    
    for value in hyp_param_to_vary:        
        # change hyperparameter that is being varied
        if hyp_param_to_vary == epsilons:
            epsilon = value
        elif hyp_param_to_vary == alpha_wds:
            alpha_wd = value
        else:
            alpha_jr = value
        
        print(f'\nBeginning training for model: models/ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}_{training_run}')

        # instantiate model and optimizer
        learning_rate = 0.01
        momentum = 0.9
        lenet = LeNet()
        optimizer = optim.SGD(lenet.parameters(), lr=learning_rate, momentum=momentum)
        lr_decayer = StepLR(optimizer, step_size=10, gamma=0.1)

        # make model CUDA enabled and define GPU/device to use
        lenet.cuda()
        device = 0
        
        # define adversary to train against if needed
        FGSM == None
        if epsilon != 0:
            FGSM = GradientSignAttack(predict=lenet, loss_fn=F.cross_entropy, 
                            eps=epsilon, clip_min=-3., clip_max=3., targeted=False)

        # for tracking training progress
        train_losses = []
        test_losses = []
        test_accuracies = []

        for epoch in range(1, n_epochs + 1):
            train(epoch, alpha_wd=alpha_wd, alpha_jr=alpha_jr, adversary=FGSM)
            test(alpha_wd=alpha_wd, alpha_jr=alpha_jr)
            lr_decayer.step()
        
        performance[f'ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}'] = (train_losses, test_losses, test_accuracies)
        torch.save(lenet.state_dict(), f'models/ep{epsilon}_wd{alpha_wd}_jr{alpha_jr}')

f = open(f'training_round_{training_round}_performance.txt','w')
f.write(str(performance))
f.close()

# to read dictionary from file:
# >>> f = open(f'training_round_{training_round}_performance.txt','w')
# >>> d = eval(f.read())


Beginning training for model: models/ep0_wd0_jr1e-07_1


NameError: name 'LeNet' is not defined